<p style="text-align: center;"><img src="https://upload.wikimedia.org/wikipedia/pt/1/1f/ITA_logo.png" alt="Resultado de imagem para logo ita" width="209" height="79" /></p>
<p style="text-align: center;"><span style="font-size: 14pt;">Instituto Tecnol&oacute;gico de Aeron&aacute;utica</span></p>
<p style="text-align: center;"><span style="font-size: 14pt;">P&oacute;s-Gradua&ccedil;&atilde;o em Engenharia Eletr&ocirc;nica e Computa&ccedil;&atilde;o</span></p>
<p style="text-align: center;"><span style="font-size: 14pt;"><strong>Orientador:</strong> Paulo Marcelo Tasinaffo</span></p>
<p style="text-align: center;"><span style="font-size: 14pt;"><strong>Co-orientador:</strong> Rodrigo Arnaldo Scarpel</span></p>
<p style="text-align: center;"><span style="font-size: 14pt;"><strong>Aluna:</strong> Bruna dos Santos Almeida</span></p>


## Import das Bibliotecas

In [1]:
import pandas as pd
import os
import time
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

from tensorflow import keras
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
#from keras.utils import to_categorical
from google.colab import drive
from sklearn.utils import class_weight

# Deep Learning Libraries
from keras.layers import Dense
from keras.models import Sequential
from keras.callbacks import EarlyStopping
from sklearn.utils import class_weight

drive.mount('/content/gdrive')

Mounted at /content/gdrive


## Import do Dataset

In [2]:
drive_path = "/content/gdrive/My Drive/Colab/Thesis/"
folder = "data/"
filename = "data_synthetic_"
folder_x_y = drive_path + folder + "train_test/"

In [3]:
data_hundred = list(range(3,37,3))
data_synthetic_list = []
count = 1
for item in data_hundred:
  dict_data = {}
  dict_data['key'] = item 

  print("Processing: "+str(count)+ " of "+str(len(data_hundred)))  
  print("Openning data train: "+str(item))
  count = count + 1
  data_train_path = os.path.join(folder_x_y, 'data_'+ str(item) + '_train.csv')
  data_train = pd.read_csv(data_train_path).drop("Unnamed: 0", axis=1)
  dict_data['train'] = data_train

  data_train_y = data_train['target']
  data_train_x = data_train.iloc[:,:-1]
  
  dict_data['train_y'] = keras.utils.to_categorical(data_train_y)
  dict_data['train_x'] = StandardScaler().fit_transform(data_train_x)
  
  #print("Openning data test: "+str(item))
  data_test_path = os.path.join(folder_x_y, 'data_'+ str(item) + '_test.csv')
  data_test = pd.read_csv(data_test_path).drop("Unnamed: 0", axis=1)
  dict_data['test'] = data_test

  dict_data['test_y'] = data_test['target']
  dict_data['test_x']  = data_test.iloc[:,:-1]
  
  #print("Salve ratio and early_stopping_monitor: "+str(item))
  non_default = len(data_train[data_train['target']==0])
  default = len(data_train[data_train['target']==1])
  ratio = float(default/(non_default+default))
  dict_data['ratio'] = ratio

  #n_cols = data_train_x.shape[1]
  dict_data['early_stopping_monitor'] = EarlyStopping(patience=10)
  class_weight = {0:ratio, 1:1-ratio}
  dict_data['class_weight'] = class_weight
  
  data_synthetic_list.append(dict_data)

Processing: 1 of 12
Openning data train: 3
Processing: 2 of 12
Openning data train: 6
Processing: 3 of 12
Openning data train: 9
Processing: 4 of 12
Openning data train: 12
Processing: 5 of 12
Openning data train: 15
Processing: 6 of 12
Openning data train: 18
Processing: 7 of 12
Openning data train: 21
Processing: 8 of 12
Openning data train: 24
Processing: 9 of 12
Openning data train: 27
Processing: 10 of 12
Openning data train: 30
Processing: 11 of 12
Openning data train: 33
Processing: 12 of 12
Openning data train: 36


# Artificial Neural Network Model

## Metrics

In [4]:
METRICS = [
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn')
]

In [5]:
COLUMNS = ['layers', 'nodes_from_1', 'nodes_from_2', 'nodes_from_3', 'iteration', 'accuracy', 'val_accuracy', 'loss', 'val_loss',
           'auc', 'val_auc','precision', 'val_precision', 'recall', 'val_recall','tp', 
           'val_tp', 'fp', 'val_fp', 'tn', 'val_tn']
describe_models = pd.DataFrame(columns = COLUMNS)

# Models

In [6]:
def model_training(item, data_hundred,  data_train_x, data_train_y, data_test_x, data_test_y, 
                   class_weight, model, data_training, n_layer, 
                   nodes_from_1, nodes_from_2, nodes_from_3):
    model_total_iter = []
    for i in range(25):
      print("Run Iteration: "+str(i+1)+" in dataset "+str(item+1))
      model_iter = {}
      model_iter['key'] = i+1

      optimizer = tf.keras.optimizers.Adadelta(learning_rate=1)
      batch_size = 2000
      EPOCHS = 200
      early_stopping_monitor = EarlyStopping(patience=10)

      model.compile(optimizer=optimizer, 
                    loss='binary_crossentropy', 
                    metrics=METRICS)
      historic = model.fit(data_train_x, 
                            data_train_y,
                            epochs=EPOCHS, 
                            validation_split=0.25, 
                            callbacks = [early_stopping_monitor], 
                            class_weight=class_weight,
                            batch_size = batch_size)
      model_iter['historic'] = historic.history

      loss_test, accuracy_test, precision_test, recall_test, auc_test, tp_test, fp_test, tn_test, fn_test = model.evaluate(data_test_x, data_test_y, batch_size=128)

      new_row = {'layers': n_layer, 
                 'nodes_from_1': nodes_from_1, 
                 'nodes_from_2': nodes_from_2, 
                 'nodes_from_3': nodes_from_3, 
                 'iteration': i+1, 
                 'dataset': data_hundred[item],
                 'accuracy': historic.history['accuracy'], 
                 'val_accuracy': historic.history['val_accuracy'],
                 'loss': historic.history['loss'], 
                 'val_loss': historic.history['val_auc'],
                 'auc': historic.history['auc'], 
                 'val_auc': historic.history['val_auc'],
                 'precision': historic.history['precision'],
                 'val_precision':historic.history['val_precision'],
                 'recall': historic.history['recall'], 
                 'val_recall': historic.history['val_recall'],
                 'tp': historic.history['tp'], 
                 'val_tp': historic.history['val_tp'], 
                 'fp': historic.history['fp'], 
                 'val_fp': historic.history['val_fp'], 
                 'tn': historic.history['tn'], 
                 'val_tn': historic.history['val_tn'],
                 'fn': historic.history['fn'],
                 'val_fn': historic.history['val_fn'],
                 'loss_test': loss_test,
                 'accuracy_test': accuracy_test,
                 'precision_test': precision_test,
                 'recall_test': recall_test,
                 'auc_test': auc_test,
                 'tp_test': tp_test,
                 'fp_test': fp_test,
                 'tn_test': tn_test,
                 'fn_test': fn_test}

      data_training = data_training.append(new_row, ignore_index=True)
        
      model_total_iter.append(model_iter)
    return model_total_iter, data_training

## 100 Features

### 1 Layer

In [7]:
def features_100_model_1_hidden_layer():
    model = tf.keras.models.Sequential()
    model.add(tf.keras.Input(shape=(100,)))
    model.add(tf.keras.layers.Dense(66))
    model.add(tf.keras.layers.Dense(2, activation="softmax"))

    model.summary()
    return model

### 2 Layers

In [8]:
def features_100_model_2_hidden_layers():
    model = tf.keras.models.Sequential()
    model.add(tf.keras.Input(shape=(100,)))
    model.add(tf.keras.layers.Dense(66))
    model.add(tf.keras.layers.Dense(44))
    model.add(tf.keras.layers.Dense(2, activation="softmax"))

    model.summary()
    return model

### 3 Layers

In [9]:
def features_100_model_3_hidden_layers():
    model = tf.keras.models.Sequential()
    model.add(tf.keras.Input(shape=(100,)))
    model.add(tf.keras.layers.Dense(66))
    model.add(tf.keras.layers.Dense(44))
    model.add(tf.keras.layers.Dense(29))
    model.add(tf.keras.layers.Dense(2, activation="softmax"))

    model.summary()
    return model

## Run Models

In [10]:
def training(item, data_hundred, n_layer, nodes_from_1, nodes_from_2, nodes_from_3, model, 
             data_synthetic_list, data_training):

    data_train_x = data_synthetic_list[item]['train_x']
    data_train_y = data_synthetic_list[item]['train_y']

    data_test_x = data_synthetic_list[item]['test_x']
    data_test_y = data_synthetic_list[item]['test_y']

    data_test_y = pd.DataFrame(data=data_test_y)
    data_test_y.rename(columns = {'0':'target'}, inplace = True)
    data_test_y[0] = 0
    data_test_y[1] = 0
    data_test_y.loc[data_test_y['target'] == 0, 0] = 1
    data_test_y.loc[data_test_y['target'] == 1, 1] = 1
    data_test_y = data_test_y.drop('target', axis = 1)

    class_weight = data_synthetic_list[item]['class_weight']

    model_total_iter, data_training = model_training(item, data_hundred, data_train_x, data_train_y, 
                                   data_test_x, data_test_y, 
                                   class_weight, model, 
                                   data_training, n_layer, 
                                   nodes_from_1, nodes_from_2, nodes_from_3)
    
    data_synthetic_list[item]['training'] = model_total_iter

    return data_synthetic_list, data_training

In [11]:
file_name = "data_training_result_100.csv"
training_result_path = drive_path + folder + file_name
training_result_path

'/content/gdrive/My Drive/Colab/Thesis/data/data_training_result_100.csv'

In [ ]:
for item in range(len(data_synthetic_list)):
  print("Run dataset: " + str(item) + " in "+str(len(data_synthetic_list)))
  try: 
    data_training = pd.read_csv(training_result_path)
  except:
    COLUMNS = ['layers', 'nodes_from_1', 'nodes_from_2', 'nodes_from_3',
               'dataset', 'iteration', 'accuracy', 'val_accuracy', 'loss', 
               'val_loss', 'auc', 'val_auc','precision', 'val_precision', 
               'recall', 'val_recall','tp', 'val_tp', 'fp', 'val_fp', 'tn', 
               'val_tn', 'fn', 'val_fn','loss_test', 'accuracy_test',
               'precision_test', 'recall_test', 'auc_test','tp_test',
               'fp_test','tn_test','fn_test']
    data_training = pd.DataFrame(columns = COLUMNS)
  
  n_layer = 1
  nodes_from_1 = 66
  nodes_from_2 = None
  nodes_from_3 = None
  model = features_100_model_1_hidden_layer()
  data_synthetic_list, data_training = training(item, data_hundred, n_layer, 
                                                nodes_from_1, nodes_from_2, 
                                                nodes_from_3, model, 
                                                data_synthetic_list, data_training)
  
  data_training.to_csv(training_result_path, index=False)

Streaming output truncated to the last 5000 lines.
3/3 [==============================] - 0s 20ms/step - loss: 0.2794 - accuracy: 0.6832 - precision: 0.6832 - recall: 0.6832 - auc: 0.7309 - tp: 3843.0000 - fp: 1782.0000 - tn: 3843.0000 - fn: 1782.0000 - val_loss: 0.6398 - val_accuracy: 0.6581 - val_precision: 0.6581 - val_recall: 0.6581 - val_auc: 0.6961 - val_tp: 1234.0000 - val_fp: 641.0000 - val_tn: 1234.0000 - val_fn: 641.0000
Epoch 3/200
3/3 [==============================] - 0s 20ms/step - loss: 0.2794 - accuracy: 0.6836 - precision: 0.6836 - recall: 0.6836 - auc: 0.7316 - tp: 3845.0000 - fp: 1780.0000 - tn: 3845.0000 - fn: 1780.0000 - val_loss: 0.6400 - val_accuracy: 0.6592 - val_precision: 0.6592 - val_recall: 0.6592 - val_auc: 0.6957 - val_tp: 1236.0000 - val_fp: 639.0000 - val_tn: 1236.0000 - val_fn: 639.0000
Epoch 4/200
3/3 [==============================] - 0s 20ms/step - loss: 0.2795 - accuracy: 0.6855 - precision: 0.6855 - recall: 0.6855 - auc: 0.7304 - tp: 3856.0000 - fp

In [ ]:
for item in range(len(data_synthetic_list)):
  print("Run dataset: " + str(item) + " in "+str(len(data_synthetic_list)))
  try: 
    data_training = pd.read_csv(training_result_path)
  except:
    COLUMNS = ['layers', 'nodes_from_1', 'nodes_from_2', 'nodes_from_3',
               'dataset', 'iteration', 'accuracy', 'val_accuracy', 'loss', 'val_loss',
               'auc', 'val_auc','precision', 'val_precision', 'recall', 
               'val_recall','tp', 'val_tp', 'fp', 'val_fp', 'tn', 'val_tn', 
               'loss_test', 'accuracy_test','precision_test', 'recall_test',
               'auc_test','tp_test','fp_test','tn_test','fn_test']
    data_training = pd.DataFrame(columns = COLUMNS)

  n_layer = 2
  nodes_from_1 = 66
  nodes_from_2 = 44
  nodes_from_3 = None
  model = features_100_model_2_hidden_layers()

  data_synthetic_list, data_training = training(item, data_hundred, n_layer, 
                                                nodes_from_1, nodes_from_2, 
                                                nodes_from_3, model, 
                                                data_synthetic_list, data_training)
  
  data_training.to_csv(training_result_path, index=False)

Streaming output truncated to the last 5000 lines.
3/3 [==============================] - 0s 20ms/step - loss: 0.2797 - accuracy: 0.6811 - precision: 0.6811 - recall: 0.6811 - auc: 0.7295 - tp: 3831.0000 - fp: 1794.0000 - tn: 3831.0000 - fn: 1794.0000 - val_loss: 0.6392 - val_accuracy: 0.6619 - val_precision: 0.6619 - val_recall: 0.6619 - val_auc: 0.6966 - val_tp: 1241.0000 - val_fp: 634.0000 - val_tn: 1241.0000 - val_fn: 634.0000
Epoch 15/200
3/3 [==============================] - 0s 21ms/step - loss: 0.2797 - accuracy: 0.6825 - precision: 0.6825 - recall: 0.6825 - auc: 0.7306 - tp: 3839.0000 - fp: 1786.0000 - tn: 3839.0000 - fn: 1786.0000 - val_loss: 0.6398 - val_accuracy: 0.6597 - val_precision: 0.6597 - val_recall: 0.6597 - val_auc: 0.6956 - val_tp: 1237.0000 - val_fp: 638.0000 - val_tn: 1237.0000 - val_fn: 638.0000
Epoch 16/200
3/3 [==============================] - 0s 21ms/step - loss: 0.2797 - accuracy: 0.6828 - precision: 0.6828 - recall: 0.6828 - auc: 0.7303 - tp: 3841.0000 - 

In [ ]:
for item in range(len(data_synthetic_list)):
  print("Run dataset: " + str(item) + " in "+str(len(data_synthetic_list)))
  try: 
    data_training = pd.read_csv(training_result_path)
  except:
    COLUMNS = ['layers', 'nodes_from_1', 'nodes_from_2', 'nodes_from_3',
               'dataset', 'iteration', 'accuracy', 'val_accuracy', 'loss', 'val_loss',
               'auc', 'val_auc','precision', 'val_precision', 'recall', 
               'val_recall','tp', 'val_tp', 'fp', 'val_fp', 'tn', 'val_tn', 
               'loss_test', 'accuracy_test','precision_test', 'recall_test',
               'auc_test','tp_test','fp_test','tn_test','fn_test']
    data_training = pd.DataFrame(columns = COLUMNS)

  n_layer = 3
  nodes_from_1 = 66
  nodes_from_2 = 44
  nodes_from_3 = 29
  model = features_100_model_3_hidden_layers()

  data_synthetic_list, data_training = training(item, data_hundred, n_layer, 
                                                nodes_from_1, nodes_from_2, 
                                                nodes_from_3, model, 
                                                data_synthetic_list, data_training)
  
  data_training.to_csv(training_result_path, index=False)